# VacationPy
---

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import geopandas as gpd
import requests
import folium
from requests.structures import CaseInsensitiveDict
import os
import holoviews as hv



# Import API key
from api_keys import geoapify_key

# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("resources/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (C),Max Temp (F)
0,0,nuku'alofa,-21.1333,-175.2000,301.34,89,100,5.66,TO,1707725623,28.19,82.742
1,1,remire-montjoly,4.9167,-52.2667,299.17,89,75,3.09,GF,1707725624,26.02,78.836
2,2,blackmans bay,-43.0167,147.3167,292.22,68,98,2.91,AU,1707725624,19.07,66.326
3,3,puerto ayora,-0.7393,-90.3518,299.01,99,8,1.34,EC,1707725625,25.86,78.548
4,4,richards bay,-28.7830,32.0377,300.31,51,21,2.96,ZA,1707725625,27.16,80.888


In [11]:
print(city_data_df.describe())

         City_ID         Lat         Lng    Max Temp    Humidity  Cloudiness  \
count  587.00000  587.000000  587.000000  587.000000  587.000000  587.000000   
mean   293.00000   17.378833   17.525486  285.995451   72.546848   54.483816   
std    169.59658   32.788889   90.203846   16.602127   22.847150   41.272353   
min      0.00000  -54.800000 -176.559700  234.740000    8.000000    0.000000   
25%    146.50000   -9.658750  -64.110450  274.285000   60.000000    8.000000   
50%    293.00000   20.460700   23.285700  292.070000   79.000000   58.000000   
75%    439.50000   46.200050   99.082500  299.430000   89.000000  100.000000   
max    586.00000   78.218600  178.441500  309.840000  100.000000  100.000000   

       Wind Speed          Date  Max Temp (C)  Max Temp (F)  
count  587.000000  5.870000e+02    587.000000    587.000000  
mean     3.758552  1.707726e+09     12.845451     55.121813  
std      2.684207  1.249713e+02     16.602127     29.883828  
min      0.000000  1.707725e+09

In [12]:
# Convert temperature from Kelvin to Celsius and Fahrenheit
city_data_df['Max Temp (C)'] = city_data_df['Max Temp'] - 273.15
city_data_df['Max Temp (F)'] = city_data_df['Max Temp (C)'] * 9/5 + 32

# Convert 'Humidity' column to numeric and change non-numeric values to NaN
city_data_df['Humidity'] = pd.to_numeric(city_data_df['Humidity'], errors='coerce')

# Drop rows with missing values in the 'Humidity' column
city_data_df = city_data_df.dropna(subset=['Humidity'])

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(
    city_data_df, 
    geometry=gpd.points_from_xy(city_data_df['Lng'], city_data_df['Lat'])
)

# Convert 'City' column and assign unique color to each category
city_categories = city_data_df['City'].astype('category')
color_mapping = dict(enumerate(city_categories.cat.codes))

# Plot the GeoDataFrame with points sized by humidity and colored by city
city_humidity = gdf.hvplot.points(
    geo=True,
    tiles='OSM',
    title='City Humidity Overview',
    frame_width=800,
    frame_height=600,
    hover_cols=['City', 'Humidity'],  
    size='Humidity',  
    color=city_categories.cat.codes,  
    cmap='Tab20',  
    line_color='black',  
    legend=True  
)


# Render the Overlay to a Matplotlib figure
fig = hv.render(city_humidity, backend='matplotlib')

# Save the figure to the resources folder
output_dir = "resources"
os.makedirs(output_dir, exist_ok=True) 
output_path = os.path.join(output_dir, "City_Humidity_World.png")
fig.savefig(output_path)

# Display
city_humidity


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (_color,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:

# Use the Pandas copy function to create a copy of the filtered DataFrame
filtered_df = city_data_df[
    (city_data_df['Humidity'] >= 40) & (city_data_df['Humidity'] <= 70) &
    (city_data_df['Max Temp (F)'] >= 72) & (city_data_df['Max Temp (F)'] <= 80) &
    (city_data_df['Wind Speed'] <= 6)
].copy()

# Reset index
filtered_df.reset_index(drop=True, inplace=True)

# Drop any rows with null values
filtered_df.dropna(inplace=True)

# Display sample data
print(filtered_df.head())




   City_ID          City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0       38   maryborough -25.5333  152.7000    299.32        70          21   
1      163  port lincoln -34.7333  135.8667    297.15        55           0   
2      202         arica -18.4750  -70.3042    296.90        68         100   
4      274       badiyah  22.4500   58.8000    297.24        58          85   
5      372  east ballina -28.8667  153.5833    298.04        65          63   

   Wind Speed Country        Date  Max Temp (C)  Max Temp (F)  
0        4.66      AU  1707725646         26.17        79.106  
1        5.46      AU  1707725735         24.00        75.200  
2        2.57      CL  1707725761         23.75        74.750  
4        5.93      OM  1707725812         24.09        75.362  
5        3.75      AU  1707725877         24.89        76.802  


In [15]:
len(filtered_df)


7

### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Max Temp (F)', 'Wind Speed']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
print(hotel_df.head())



           City Country      Lat       Lng  Humidity  Max Temp (F)  \
0   maryborough      AU -25.5333  152.7000        70        79.106   
1  port lincoln      AU -34.7333  135.8667        55        75.200   
2         arica      CL -18.4750  -70.3042        68        74.750   
4       badiyah      OM  22.4500   58.8000        58        75.362   
5  east ballina      AU -28.8667  153.5833        65        76.802   

   Wind Speed Hotel Name  
0        4.66             
1        5.46             
2        2.57             
4        5.93             
5        3.75             


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
import requests
from requests.structures import CaseInsensitiveDict

#URL with the API key
url = "https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=rect%3A10.716463143326969%2C48.755151258420966%2C10.835314015356737%2C48.680903341613316&limit=20&apiKey={}".format(geoapify_key)

# Set headers
headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

# Make the request
try:
    resp = requests.get(url, headers=headers)
    print("Status code:", resp.status_code)
    print("Response:", resp.text)
except Exception as e:
    print("Error:", e)

Status code: 200
Response: {"type":"FeatureCollection","features":[{"type":"Feature","properties":{"name":"Hotel M & S garni","country":"Germany","country_code":"de","state":"Bavaria","county":"Landkreis Donau-Ries","city":"Donauwörth","postcode":"86609","suburb":"Parkstadt","street":"Andreas-Mayr-Straße","housenumber":"11","lon":10.7886113536589,"lat":48.73404755,"formatted":"Hotel M & S garni, Andreas-Mayr-Straße 11, 86609 Donauwörth, Germany","address_line1":"Hotel M & S garni","address_line2":"Andreas-Mayr-Straße 11, 86609 Donauwörth, Germany","categories":["accommodation","accommodation.hotel","building","building.accommodation","internet_access","internet_access.free"],"details":["details","details.contact","details.facilities"],"datasource":{"sourcename":"openstreetmap","attribution":"© OpenStreetMap contributors","license":"Open Database Licence","url":"https://www.openstreetmap.org/copyright","raw":{"fax":"+49 906 23986","name":"Hotel M & S garni","email":"info@hotel-ms.de","p

In [18]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters
limit = 1 

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params = {
        "filter": f"circle:{longitude},{latitude},{radius}",
        "bias": f"proximity:{longitude},{latitude}"
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Construct the URL with the API key, parameters, and limit
    url = f"{base_url}?categories=accommodation.hotel&apiKey={geoapify_key}&limit={limit}"
    
    # Set headers
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"
    
    try:
        # Make an API request using the params dictionary
        resp = requests.get(url, headers=headers, params=params)
        
        # Convert the API response to JSON format
        data = resp.json()
        
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        if "features" in data and data["features"]:
            hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
        # Log the search results
        print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    except Exception as e:
        print(f"Error occurred while searching for hotels in {hotel_df.loc[index, 'City']}: {e}")

# Display
print(hotel_df)


Starting hotel search
maryborough - nearest hotel: Post Office Hotel
port lincoln - nearest hotel: Boston Hotel
arica - nearest hotel: Hotel Andalucía 
badiyah - nearest hotel: Al Areen Hotel Apartments
east ballina - nearest hotel: Ballina Beach Resort
lethem - nearest hotel: Hotel Amazonas
thai binh - nearest hotel: An Thai Hotel
           City Country      Lat       Lng  Humidity  Max Temp (F)  \
0   maryborough      AU -25.5333  152.7000        70        79.106   
1  port lincoln      AU -34.7333  135.8667        55        75.200   
2         arica      CL -18.4750  -70.3042        68        74.750   
4       badiyah      OM  22.4500   58.8000        58        75.362   
5  east ballina      AU -28.8667  153.5833        65        76.802   
6        lethem      GY   3.3803  -59.7968        68        76.334   
7     thai binh      VN  20.4500  106.3333        56        72.824   

   Wind Speed                 Hotel Name  
0        4.66          Post Office Hotel  
1        5.46      

In [20]:
# DataFrame
gdf = gpd.GeoDataFrame(
    hotel_df, 
    geometry=gpd.points_from_xy(hotel_df['Lng'], hotel_df['Lat'])
)

# Plot
vacation_destinations = gdf.hvplot.points(
    geo=True,
    tiles='OSM',
    title='Sunshine Escapes Map',
    frame_width=800,
    frame_height=600,
    hover_cols=['City', 'Country', 'Hotel Name', 'Max Temp (F)', 'Wind Speed'],  
    size='Humidity',  
    c='City', 
    cmap='Pastel1',  
    line_color='black',  
    legend=True  
)

# Render the Overlay to a Matplotlib figure
fig = hv.render(vacation_destinations, backend='matplotlib')

# Save the figure to the resources folder
output_dir = "resources"
os.makedirs(output_dir, exist_ok=True) 
output_path = os.path.join(output_dir, "Sunshine_Escapes_Map.png")
fig.savefig(output_path)

# Display
vacation_destinations


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,Country,Hotel Name,Max Temp (F),Wind Speed)